#Интерполяция

Задача:
1. Восстановить пропуски в данных с метеостанции с помощью линейной и квадратичной интерполяции.
2. Реализовать алгоритмы линейной и квадратичной интерполяции.

In [1]:
import pandas as pd
import numpy as np

def linear_interpolation(data, row, col):

    left_col = col - 1
    right_col = col + 1

    while left_col >= 0 and pd.isnull(data.iloc[row, left_col]):
        left_col -= 1
    while right_col < data.shape[1] and pd.isnull(data.iloc[row, right_col]):
        right_col += 1
    if left_col == -1:
        left_col += 1
        while left_col < data.shape[1] and pd.isnull(data.iloc[row, left_col]):
            left_col += 1
            right_col = left_col + 1
            if not pd.isnull(data.iloc[row, left_col]) and not pd.isnull(data.iloc[row, right_col]):
                break
    if right_col == data.shape[1]:
        right_col -= 1
        while right_col > 0 and pd.isnull(data.iloc[row, right_col]):
            right_col -= 1
            left_col = right_col - 1
            if not pd.isnull(data.iloc[row, left_col]) and not pd.isnull(data.iloc[row, right_col]):
                break

    if left_col >= 0 and right_col < data.shape[1]:
        value = data.iloc[row, left_col] + (float(data.iloc[row, right_col]) - float(data.iloc[row, left_col])) * (col - left_col) / (right_col - left_col)
        return round(value, 2)

def quadratic_interpolation(data, row, col):

    left_left_col = col - 2
    left_col = col - 1
    right_col = col + 1
    while left_left_col > 1 and np.isnan(data.iloc[row, left_left_col]):
        left_left_col = left_col - 1
    while left_col > 0 and np.isnan(data.iloc[row, left_col]):
        left_col -= 1
    while right_col < data.shape[1] and np.isnan(data.iloc[row, right_col]):
        right_col += 1
    if left_left_col == -2:
        left_left_col += 2
        left_col = left_left_col + 1
        right_col = left_col + 1
        while left_left_col < data.shape[1] -2 and pd.isnull(data.iloc[row, left_left_col]):
            left_left_col += 1
            left_col = left_left_col + 1
            right_col = left_col + 1
        while left_col < data.shape[1] -1 and pd.isnull(data.iloc[row, left_col]):
            left_col = left_left_col + 1
            right_col = left_col + 1
        while right_col < data.shape[1] and pd.isnull(data.iloc[row, right_col]):
            right_col = left_col + 1
    if left_left_col == -1:
        left_left_col += 1
        left_col = left_left_col + 1
        right_col = left_col + 1
        while left_left_col < data.shape[1] -2 and pd.isnull(data.iloc[row, left_left_col]):
            left_left_col += 1
            left_col = left_left_col + 1
            right_col = left_col + 1
        while left_col < data.shape[1] -1 and pd.isnull(data.iloc[row, left_col]):
            left_col += 1
            right_col = left_col + 1
        while right_col < data.shape[1] and pd.isnull(data.iloc[row, right_col]):
            right_col = left_col + 1
    if right_col == data.shape[1]:
        right_col -= 1
        left_col = right_col - 1
        left_left_col = left_col - 1
        while right_col > 2 and pd.isnull(data.iloc[row, right_col]):
            right_col -= 1
            left_col = right_col - 1
            left_left_col = left_col - 1
        while left_col > 1 and pd.isnull(data.iloc[row, left_col]):
            left_col = right_col - 1
            left_left_col = left_col - 1
        while left_left_col > 0 and pd.isnull(data.iloc[row, left_left_col]):
            left_left_col = left_col - 1

    if left_left_col >= 0 and right_col < data.shape[1]:
        x = col
        x1 = left_left_col
        x2 = left_col
        x3 = right_col
        y1 = data.iloc[row, left_left_col]
        y2 = data.iloc[row, left_col]
        y3 = data.iloc[row, right_col]

        out = y1*((x-x2)*(x-x3))/((x1-x2)*(x1-x3)) + y2*((x-x1)*(x-x3))/((x2-x1)*(x2-x3)) + y3*((x-x1)*(x-x2))/((x3-x1)*(x3-x2))

        return round(out, 2)

################### Температура ######################

data = pd.read_csv('temp.csv', header=2, usecols=list(range(0, 25)), index_col=0, decimal=",")
data1 = data.apply(pd.to_numeric, errors='coerce')

for row in range(data1.shape[0]):
    for col in range(data1.shape[1]):
        if pd.isnull(data1.iloc[row, col]):
            data1.iloc[row, col] = linear_interpolation(data1, row, col)
data1.to_csv('linear_interpolate_temperature_data.csv')
print("Файл с линейной интерполяцией температуры - linear_interpolate_temperature_data.csv")

data2 = data.apply(pd.to_numeric, errors='coerce')
for row in range(data2.shape[0]):
    for col in range(data2.shape[1]):
        if pd.isnull(data2.iloc[row, col]):
            data2.iloc[row, col] = quadratic_interpolation(data2, row, col)
data2.to_csv('quad_interpolate_temperature_data.csv')
print("Файл с квадратичной интерполяцией температуры - quad_interpolate_temperature_data.csv")

################## Точка росы ###################

data = pd.read_csv('dew_point.csv', header=2, usecols=list(range(0, 25)), index_col=0, decimal=",")
data1 = data.apply(pd.to_numeric, errors='coerce')
data1 = data1.iloc[0:31]
for row in range(data1.shape[0]):
    for col in range(data1.shape[1]):
        if pd.isnull(data1.iloc[row, col]):
            data1.iloc[row, col] = linear_interpolation(data1, row, col)
data1.to_csv('linear_interpolate_dew_point_data.csv')
print("Файл с линейной интерполяцией точки росы - linear_interpolate_dew_point_data.csv")

data2 = data.apply(pd.to_numeric, errors='coerce')
data2 = data2.iloc[0:31]
for row in range(data2.shape[0]):
    for col in range(data2.shape[1]):
        if pd.isnull(data2.iloc[row, col]):
            data2.iloc[row, col] = quadratic_interpolation(data2, row, col)
data2.to_csv('quad_interpolate_dew_point_data.csv')
print("Файл с квадратичной интерполяцией точки росы - quad_interpolate_dew_point_data.csv")


################## Атмосферное давление ###################

data = pd.read_csv('atm_pressure.csv', header=1, usecols=list(range(0, 25)), index_col=0, decimal=",")
data1 = data.apply(pd.to_numeric, errors='coerce')
data1 = data1.iloc[0:31]
for row in range(data1.shape[0]):
    for col in range(data1.shape[1]):
        if pd.isnull(data1.iloc[row, col]):
            data1.iloc[row, col] = linear_interpolation(data1, row, col)
data1.to_csv('linear_interpolate_atm_pressure_data.csv')
print("Файл с линейной интерполяцией атмосферного давления - linear_interpolate_atm_pressure_data.csv")

data2 = data.apply(pd.to_numeric, errors='coerce')
data2 = data2.iloc[0:31]
for row in range(data2.shape[0]):
    for col in range(data2.shape[1]):
        if pd.isnull(data2.iloc[row, col]):
            data2.iloc[row, col] = quadratic_interpolation(data2, row, col)
data2.to_csv('quad_interpolate_atm_pressure_data.csv')
print("Файл с квадратичной интерполяцией атмосферного давления - quad_interpolate_atm_pressure_data.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'temp.csv'